In [1]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, hilbert
import json
from tqdm import tqdm
import h5py
import multiprocessing

In [2]:
def load_data(root, subject):
    data = pd.read_csv(f'{root}/{subject}/{subject}_data.csv', header=None)
    with open(f'{root}/{subject}/{subject}_param.json', 'r') as file:
        param = json.load(file)
    if len(param['channel_id_labels']) == data.shape[1]:    
        print(f'Loaded {data.shape[0]} datapoints and {len(param.keys())} parameters for {data.shape[1]} channels')
    else:
        print('Error loading data')
    return data, param

In [30]:
def extract_params(param, datashape):
    Nch = datashape[1]
    sample_rate = param['sample_rate']
    tperm = param['tperm']
    Nperm = param['Nperm']
    sbin = round(param['tbin'] * sample_rate)
    sstp = round(param['tstp'] * sample_rate)
    print(datashape[0])    
    print(datashape[1])
    rT = np.arange(0, datashape[0]-sbin, sstp)
    Nt = len(rT)
    t = (rT + sbin / 2) / sample_rate
    rP = np.column_stack((param['fP'][:-1], param['fP'][1:]))
    rA = np.column_stack((param['fA'][:-1], param['fA'][1:]))
    NP = rP.shape[0]
    NA = rA.shape[0]
    edges = np.arange(-np.pi, np.pi+param['delta'], param['delta'])
    x = edges[:-1] + param['delta'] / 2
    Nx = len(x)
    return Nch, sample_rate, tperm, Nperm, sbin, sstp, rT, Nt, t, rP, rA, NP, NA, edges, x, Nx

In [5]:
def save_results(PACmi, root, subject):
    PACmi_transposed = np.transpose(PACmi, (4, 3, 2, 1, 0))
    filename = f"{root}/{subject}/{subject}_PACmi.h5"
    with h5py.File(filename, 'w') as f:
        # Save the array to the file
        f.create_dataset('PACmi', data=PACmi_transposed)

In [4]:
root = '/Users/tnl/matlab/sEEG_proc'
subject = 'HUP241_RID890'
data, param = load_data(root, subject)
Nch, sample_rate, tperm, Nperm, sbin, sstp, rT, Nt, t, rP, rA, NP, NA, edges, x, Nx = extract_params(param, data.shape)

Loaded 2914304 datapoints and 10 parameters for 36 channels


In [18]:
tmi = np.zeros((NP, NA, Nt))
mrl = np.zeros((NP, NA, Nt))
mu = np.zeros((NP, NA, Nt))
tmip = np.zeros((NP, NA, Nt))
mrlp = np.zeros((NP, NA, Nt))
iP = 0
bP, aP = butter(2, rP[iP, :] / (sample_rate / 2), btype='band')
print(bP)
print(aP)

[ 6.14181825e-08  0.00000000e+00 -1.22836365e-07  0.00000000e+00
  6.14181825e-08]
[ 1.         -3.99927705  5.99785328 -3.99787539  0.99929916]


In [20]:
ich = 0
P = np.angle(hilbert(filtfilt(bP, aP, data[ich].to_numpy())))
print(P.shape)
print(P[0])

(2914304,)
-1.1039985291170669


In [36]:
Pbin = np.zeros((sbin, Nt), dtype=int)
iT = 0
t1 = rT[iT]
t2 = rT[iT] + sbin
cP = P[t1:t2]
Pbin[:, iT] = np.digitize(cP, edges)

In [44]:
iA = 0
bA, aA = butter(2, rA[iA, :] / (sample_rate / 2), btype='band')
print(bA, aA)

[ 2.40818133e-06  0.00000000e+00 -4.81636266e-06  0.00000000e+00
  2.40818133e-06] [ 1.         -3.99504064  5.98569817 -3.98627303  0.99561558]


In [46]:
A = np.abs(hilbert(filtfilt(bA, aA, data[ich].to_numpy())))
print(A.shape)
print(A[0])

(2914304,)
8.03716037728431


In [47]:
iT = 0
t1 = rT[iT]
t2 = rT[iT] + sbin
cA = A[t1:t2]
cAm = np.zeros(Nx)

In [56]:
for jj in range(Nx):
    cAm[jj] = np.mean(cA[Pbin[:, iT] == jj])
cAm

array([        nan, 18.60148554, 18.57019077, 18.55491346, 18.55011831,
       18.52758127, 18.5256682 , 18.53360616, 18.6002194 , 18.66921464,
       18.74652309, 18.75860608, 18.71290433, 18.68013297, 18.65824344,
       18.6609928 ])

In [57]:
for jj in range(Nx):
    cAm[jj] = np.mean(cA[Pbin[:, iT] == jj+1])
cAm

array([18.60148554, 18.57019077, 18.55491346, 18.55011831, 18.52758127,
       18.5256682 , 18.53360616, 18.6002194 , 18.66921464, 18.74652309,
       18.75860608, 18.71290433, 18.68013297, 18.65824344, 18.6609928 ,
       18.64698748])